# Planilha de faturamento médico para estudos do CPC HBR

In [5]:
import numpy as np
import pandas as pd
from pandas_ods_reader import read_ods
import datetime as dt
import xlsxwriter
import io
import zipfile


In [26]:
#algumas funções importantes
def gerar_zipfile(arquivos):
    mem_zip = io.BytesIO()
    with zipfile.ZipFile(mem_zip, mode='w', compression=zipfile.ZIP_DEFLATED) as zf:
        for arquivo in arquivos:
            zf.writestr('nome', arquivo)
    return mem_zip.getvalue()

In [10]:
def receber_planilha():
    dadosPagamento = read_ods('Honorarios_medicos_estudos_DASA.ods', 'atendimentos_realizados')
    dadosMedicos = read_ods('Honorarios_medicos_estudos_DASA.ods', 'dados_medicos')
    dadosMedicosRelevantes = dadosMedicos[['Medico', 'CPF', 'Nome_CNPJ', 'CNPJ']]
    dadosPagamento = dadosPagamento[dadosPagamento['Pago'] == 'NAO']
    dadosPagamento['Mes de Atendimento'] = pd.to_datetime(dadosPagamento['Data_de_realizacao']).dt.to_period('M')
    dadosPagamento.join(dadosMedicosRelevantes.set_index('Medico'), on="Medico", how='inner')
    #retorna um dataframe pandas
    return dadosPagamento

In [3]:
    #capturando nomes dos médicos
medicos = pd.unique(dadosPagamento['Medico'])
medico = "MARINA DE SOUSA SILVA"
#colunas que serao usadas para pagamento
colunas_pagamento = dadosPagamento[['Medico', 'Centro', 'Estudo', 'tipo_atendimento','Mes de Atendimento', 'Valor']]
df_pagamento = colunas_pagamento['Medico'] == medico
outro = colunas_pagamento[df_pagamento]
#descricao = dadosPagamento[[dadosPagamento['Medico'] == medico]]
agrupado = outro.groupby(['Medico', 'Centro', 'Estudo', 'tipo_atendimento','Mes de Atendimento']).count().reset_index()
#agrupado['Quantidade'] = agrupado["count"]*agrupado["Valor"]
#df = df['Valor'].append(df['Valor'].agg(['sum']))
#nome_arquivo = 'Horario_medico__' + medico + '.xls'
agrup2 = agrupado
agrupado.rename(columns={'Valor':'Quantidade'}, inplace=True)
agrupado["Valor"] = dadosPagamento["Valor"]
agrupado['Total'] = agrupado['Valor'] * agrupado['Quantidade']
agrupado

,Medico,Centro,Estudo,tipo_atendimento,Mes de Atendimento,Quantidade,Valor,Total
0,MARINA DE SOUSA SILVA,HBR,ACTIV-1-IM,Atendimento Internação,2021-10,11,100.0,1100.0
1,MARINA DE SOUSA SILVA,HBR,ACTIV-1-IM,Atendimento Presencial,2021-01,1,500.0,500.0
2,MARINA DE SOUSA SILVA,HBR,ACTIV-1-IM,Atendimento Presencial,2021-10,1,100.0,100.0
3,MARINA DE SOUSA SILVA,HBR,ACTIV-1-IM,Atendimento Presencial,2021-11,3,100.0,300.0
4,MARINA DE SOUSA SILVA,HBR,ACTIV-1-IM,Recrutamento,2021-09,3,100.0,300.0
5,MARINA DE SOUSA SILVA,HBR,ACTIV-1-IM,Telemedicina,2021-10,8,500.0,4000.0
6,MARINA DE SOUSA SILVA,HBR,ACTIV-1-IM,Telemedicina,2021-11,5,100.0,500.0
7,MARINA DE SOUSA SILVA,HBR,ATTRACT-3,Consentimento,2021-02,1,100.0,100.0


In [19]:

def gerar_planilhas(dadosPagamento):
    #data de hoje
    hoje = dt.date.today()
    # inicializando array de planilhas
    planilhas = []
    #capturando nomes dos médicos
    medicos = pd.unique(dadosPagamento['Medico'])
    #colunas que serao usadas para pagamento
    colunas_pagamento = dadosPagamento[['Medico', 'Centro', 'Estudo', 'tipo_atendimento','Mes de Atendimento', 'Valor']]
    #loop para filtro de dados
    for medico in medicos:
        df_pagamento = colunas_pagamento['Medico'] == medico
        filtrado = colunas_pagamento[df_pagamento]
        agrupado = filtrado.groupby(['Medico', 'Centro', 'Estudo', 'tipo_atendimento','Mes de Atendimento']).count().reset_index()
        agrupado.rename(columns={'Valor':'Quantidade'}, inplace=True)
        agrupado["Valor"] = dadosPagamento["Valor"]
        agrupado['Total'] = agrupado['Valor'] * agrupado['Quantidade']
        nome_arquivo = 'Honorario_medico_' + medico + '_' + str(hoje) + '.xls'
        
        #workbook = xlsxwriter.Workbook(nome_arquivo)
        #worksheet_pagar = workbook.add_worksheet('Pagar')

        #for i in range(0,agrupado.shape[0]):
        #    worksheet_pagar.write('A'+str(i), str(agrupado.iloc[i:1]))
        #workbook.close()


        #inicializando buffer
        buffer = io.BytesIO()
        #salvando dados em planilhas
        with pd.ExcelWriter(buffer) as writer:  
            agrupado.to_excel(writer, sheet_name='Pagar')
            filtrado.to_excel(writer, sheet_name='O que está sendo pago')
        #adicionando a planilha no array    
        planilhas.append(buffer.getvalue())
        buffer.close()
    
    return planilhas
        

  



In [27]:
dados = receber_planilha()
planilhas = gerar_planilhas(dados)
zip = gerar_zipfile(planilhas)
zip.


b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xb7q\xd6T\xae(Yz\xdb\x19\x00\x00\xa5\x1c\x00\x00\x04\x00\x00\x00nome}y\x05T\x15m\xd7\xf6\xa1;\x0e!q\xe8\x12\xa4\xf1\x00\x02\xd2p8\x944\x88HH\x1d\xba\x1b\xa4K\xba\xbb\xbb\x11$E\xe9.\xe9\x0e\t\xe9N%\x7f|\x9e\xff[\xfe\xf2\xbd\xef?\xb3f\xee\x99\xb9\xd7\xbef\xd6\xb5\xf7\xbaf\xef}+\xca" \x12\x00\x00\x00\xd4\x87C\x18\xa0\xd9\xf4\xc5Z\x15\x0e\x00 B\x02\x00\xf0\x1f\x9eh\x89[[9\x18Y9\xe8\xaa\xba\xda\x18\xd9k\xb3\xb9XZt\xc5\rX\xb5s\xe0J^\xde\x86\xf4\xce?\xe3\xf9XT\x12\xd2\x9a\x13i\x14\xaaz\x05W!H\xfd$\xc2\xd2\x15\x19\xddK\xf7)\xdc\x84:+\x8e\xcf\x84|\x99\xcb\xc1\x82}O\xf5K\xb2J\xb3TV\xea\x9c\xe3\n\xbd\xf7K\x06$/~\xd4\xda~\xf2\xfd*\x9fy\xa2\x95@-]\x98\x7f\x8e\x8d^\x87l\xb86:i\xf3\x19\xdb[\xd3\xcc\xe9\xf5\xc7\x10\xc6\x054\x9f\x12\xdf74\x97poK\xe2\x84\xa6b\xc6\xac\xd3>PM\xbe\x1d\x16\xc1t\xfb\n\xca\xfeBk\x13\xa7\xb2\xd6\x06\xe9@a\xfc\xd1\xae|\xd1 3\xaf\x1d\x85\xfaD\xd3i\x19\r\xa4\xdef\x88:\xc5(]\xcc\x96\xf0\xea>k\x1d\x04\xaf\x90^\x8e\xcc\x8c\x7f\xc4\x8f\x96\xf43~D\